# CNN Architectures
- typical architecture: stack a few conv. layers with each followed followed by a ReLU layer, then a pooling layer, then more conv. layers w/ ReLU layer after, then another pooling layer, and so on
- Image gets smaller and smaller but also gets deeper and deeper (more feature maps) as it progresses down the network
- <img src="images/CNNArchitecture.jpeg" width=500/> typical CNN architecture
- **Don't use large kernel sizes (e.g. 5x5) for conv. layers. Instead use two smaller layers (e.g. 3x3) because it uses fewer parameters, has fewer computations, and has better performance**
    - Exception on the first conv. layers because it will reduce the spatial dimensions without losing much information

In [1]:
from tensorflow.keras.datasets import fashion_mnist

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2)

#normalize values
X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_val = (X_val - X_mean) / X_std
X_test = (X_test - X_mean) / X_std


X_train = X_train[..., np.newaxis] # newaxis puts all grayscale values into an array of size 1 so that the shape will be [height, width, channel]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [3]:
# # Import mlcompute module to use the optional set_mlc_device API for device selection with ML Compute.
# from tensorflow.python.compiler.mlcompute import mlcompute

# # Select CPU device.
# mlcompute.set_mlc_device(device_name='cpu') # Available options are 'cpu', 'gpu', and 'any'.

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

model = Sequential([
    Conv2D(filters=64, kernel_size=7, activation="relu", padding="same", input_shape=[28, 28, 1]), # account for grayscale channel
    MaxPool2D(pool_size=2),
    #use two conv layers w/ small kernel size (3X3) 
    Conv2D(filters=128, kernel_size=3, activation="relu", padding="same"),
    Conv2D(filters=128, kernel_size=3, activation="relu", padding="same"),
    MaxPool2D(pool_size=2),
    Conv2D(filters=256, kernel_size=3, activation="relu", padding="same"),
    Conv2D(filters=256, kernel_size=3, activation="relu", padding="same"),
    MaxPool2D(pool_size=2),
    #Flatten before feeding outptus to the regular feed-forward portion of the network
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(10, activation="softmax")
])

In [8]:
model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=1, validation_data=(X_val, y_val))

1500/1500 [==============================] - 131s 87ms/step - loss: 0.7746 - accuracy: 0.7152 - val_loss: 0.4978 - val_accuracy: 0.8122


In [9]:
score = model.evaluate(X_test, y_test)

313/313 [==============================] - 9s 27ms/step - loss: 0.5236 - accuracy: 0.7989


In [10]:
score

[0.5235874056816101, 0.7989000082015991]